In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180


%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("..") # Adds higher directory to python modules path.
import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
import time
import warnings
warnings.filterwarnings("ignore")
from locanmf import LocaNMF
import postprocess

Autosaving every 180 seconds


In [2]:
###########################################
res = None

spatial = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy')
print (spatial.shape)

temporal = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
temporal = temporal.transpose(2,0,1)
print (temporal.shape)

res = np.matmul(temporal, spatial)
print (res.shape)
print ("DONE")



(7, 16384)
(1801, 132, 7)
(1801, 132, 16384)
DONE


In [3]:
########################################
res2 = res.transpose(1,0,2)
print (res2.shape)

res2 = res2.reshape(res2.shape[0], 
                    res2.shape[1], 
                    128, 128)
print (res2.shape)

res2_mean = res2.mean(0)



(132, 1801, 16384)
(132, 1801, 128, 128)


In [7]:
from tqdm import trange
#

print (res2_mean.shape)
res2_half = res2_mean[:res2_mean.shape[0]//2]
print (res2_half.shape)
    
# 
mu = 0
sigma = 0.05

# 
res_sim = np.zeros((132, res2_half.shape[0],
                    res2_half.shape[1],
                    res2_half.shape[2]), 'float32')

# 
for p in trange(res_sim.shape[0]):
    noise = np.random.normal(mu, sigma, size=res2_half.shape) 
    temp = res2_half + noise
    res_sim[p] = temp
    
np.save('/home/cat/res_sim'+str(sigma)+'.npy', res_sim)


  0%|          | 0/132 [00:00<?, ?it/s]

(1801, 128, 128)
(900, 128, 128)


100%|██████████| 132/132 [01:00<00:00,  2.19it/s]


In [3]:
##############################
# load PCA pbject
import pickle as pk
fname_pca = '/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca.pkl'
file = open(fname_pca, 'rb')
pca = pk.load(file)






In [5]:
# res_sim = np.load('/home/cat/res_sim.npy')
# print (res_sim.shape)


(132, 900, 128, 128)


In [10]:
X = res_sim.reshape(132*900, 128*128)
print ("X: ", X.shape)

time_filters = pca.transform(X)#[:,:nComp]

print (time_filters.shape)




X:  (118800, 16384)
(118800, 1500)


In [15]:
random = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')
print (random.shape)

np.save('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95_firsthalf.npy',
       random[:,:,:900])

(132, 7, 1801)


In [16]:
##
trials = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print (trials.shape)

#
nComp = 7
res_pca = time_filters.reshape(132,900,-1)[:,:,:7].transpose(0,2,1)
print (res_pca.shape)

fname_out = '/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_firsthalf.npy'
np.save(fname_out.replace('trial','simulated'),
       res_pca)



(132, 7, 1801)
(132, 7, 900)


In [113]:
# #
# width = 0.00001
# bins = np.arange(-0.02,0.02,width)
# f = res2_half.flatten()

# # 
# print ("f: ", f.shape)
# idx = np.where(np.abs(f)<0.000001)[0]
# print ("idx: ", idx.shape)
# f[idx]=np.nan
# idx = np.where(np.isnan(f))[0]
# print (idx.shape)

# # 
# y = np.histogram(f, 
#                  bins=bins)

# # 
# plt.plot(y[1][1:],y[0])
# plt.show()

f:  (14745600,)
idx:  (3528646,)
(3528646,)


In [35]:
# 
from sklearn.preprocessing import StandardScaler
from sklearn import svm


def run_10fold(k,
               idx,
               trials_window_flat,
               ):

    #
    idx_test = idx[k]
    X_test = trials_window_flat[:,idx_test]
    y_test = []
    for f in range(X_test.shape[0]):
        y_test.append(np.zeros(X_test[f].shape[0])+f)
        
    # stack the data
    y_test = np.hstack(y_test)
    X_test = X_test.reshape(-1, X_test.shape[2])
    #print ("X_test: ", X_test.shape, " y_test: ", y_test.shape)

    #
    idx_train = np.delete(np.arange(trials_window_flat.shape[1]), idx[k])
    train = trials_window_flat[:,idx_train]
    # print ("Train: ", train.shape)

    # loop over all features/body parts and generate labels
    y_train = []
    X_train = []
    for f in range(train.shape[0]):
        y_train.append(np.zeros(train[f].shape[0])+f)
        X_train.append(train[f])

    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)
    #print ("X_train: ", X_train.shape, " y_train: ", y_train.shape)

    # STANDARDIZE DATA
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #
    # Fit SVM/Classifier

    acc, svm_coef = run_svm_multi_variate(X_train, y_train, X_test, y_test)
    
    
    return acc

def run_svm_multi_variate(X_train, y_train, X_test, y_test):

    #linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
    # s = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
    #poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
    s = svm.SVC(kernel='sigmoid',
    #s = svm.SVC(kernel='linear',
                C=1,
                decision_function_shape='ovo').fit(X_train, y_train)

    # retrieve the accuracy and print it for all 4 kernel functions
    accuracy = s.score(X_test, y_test)
    prediction = s.predict(X_test)

    #
    # print ("s.coef_ ", s.coef_.shape, X_train.shape)
    return accuracy, prediction


# 
# load pca data
trials = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_simulated_ROItimeCourses_30sec_pca_0.95_firsthalf.npy')
random = np.load('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95_firsthalf.npy')
all_trials = np.array((trials, random))
print ("stacked trials : ", all_trials.shape)

#    
from tqdm import trange
import parmap
acc_array = []
for t in trange(0,900,1):
    t = 0
    window = 30
    t1 = t
    t2 = t + window

    # 
    trials_window = all_trials[:,:,t1:t2]
    #print ("trials full in: ", all_trials.shape, "trials window: ", trials_window.shape)

    trials_window_flat = trials_window.reshape(trials_window.shape[0],
                                               trials_window.shape[1],
                                               -1)
    #print (trials_window_flat.shape)

    # 10-fold split; should we randomize?
    idx = np.array_split(np.random.choice(np.arange(trials_window_flat.shape[1]),
                                          trials_window_flat.shape[1], replace=False),
                         10)

    # 
    ks = np.arange(10)
    if False:
        
        acc = parmap.map(run_10fold, ks,
                  idx, 
                  trials_window_flat,
                  pm_processes=10)
    else:
        acc= []
        for k in range(10):
            acc.append(run_10fold(k,
                       idx,
                       trials_window_flat,
                      ))

    acc_array.append(acc)

# 
temp = np.array(acc_array)
np.save('/media/cat/1TB/yuki/yongxu/locaNMF_data/data/IJ1/Mar3/svm_acc_simulated_0.005noise.npy', temp)

  0%|          | 0/900 [00:00<?, ?it/s]

stacked trials :  (2, 132, 7, 900)


100%|██████████| 900/900 [13:23<00:00,  1.12it/s]


In [42]:
# print (temp.shape)

mean = temp.mean(1)
std = np.std(temp,1)

t = np.arange(-900,0,1)/30
plt.plot(t,mean)
plt.ylim(0.4,1.0)
plt.show()